## Section 1

In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

# Load the data
file_path = '/Users/tangjiahong/Dropbox/HW_statistics/finalexam/GiftCards6.xlsx'
data = pd.read_excel(file_path)

# Encode categorical variables
data['Urban'] = LabelEncoder().fit_transform(data['Urban'])
data['Male'] = LabelEncoder().fit_transform(data['Male'])

# Define the predictors and the response variable
X = data[['Income_1000', 'Urban', 'Male']]
y = data['Spending']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the regression model
model1 = sm.OLS(y, X).fit()

# Display the model summary
print(model1.summary())


                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.391
Model:                            OLS   Adj. R-squared:                  0.385
Method:                 Least Squares   F-statistic:                     67.61
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           8.48e-34
Time:                        15:57:43   Log-Likelihood:                -2286.2
No. Observations:                 320   AIC:                             4580.
Df Residuals:                     316   BIC:                             4596.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -483.3711     83.618     -5.781      

In [2]:
# F-test
f_test = model1.f_test("Income_1000 = Urban = Male = 0")
print(f"F-test result:\n{f_test}")

# Confidence Interval for Male coefficient
ci_male = model1.conf_int().loc['Male']
print(f"95% Confidence Interval for Male coefficient: {ci_male}")

# Coefficient of determination (R^2)
r_squared = model1.rsquared
print(f"R-squared: {r_squared}")

# Estimate for an urban male with an annual income of $45,000
income = 45
urban = 1
male = 1
estimated_spending = model1.predict([1, income, urban, male])[0]
print(f"Estimated Spending: {estimated_spending}")

# Interaction Model
data['Urban_Male'] = data['Urban'] * data['Male']
X_interaction = data[['Income_1000', 'Urban', 'Male', 'Urban_Male']]
X_interaction = sm.add_constant(X_interaction)
model2 = sm.OLS(y, X_interaction).fit()
print(model2.summary())

# Quadratic Model
data['Income_1000_squared'] = data['Income_1000'] ** 2
X_quadratic = data[['Income_1000', 'Income_1000_squared', 'Urban', 'Male']]
X_quadratic = sm.add_constant(X_quadratic)
model3 = sm.OLS(y, X_quadratic).fit()
print(model3.summary())


F-test result:
<F test: F=67.60855766196423, p=8.475083804828367e-34, df_denom=316, df_num=3>
95% Confidence Interval for Male coefficient: 0      1.588565
1    138.958963
Name: Male, dtype: float64
R-squared: 0.39093222164332087
Estimated Spending: 134.809768629208
                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.392
Model:                            OLS   Adj. R-squared:                  0.384
Method:                 Least Squares   F-statistic:                     50.78
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           5.75e-33
Time:                        15:57:53   Log-Likelihood:                -2286.0
No. Observations:                 320   AIC:                             4582.
Df Residuals:                     315   BIC:                             4601.
Df Model:                           4                                         
Covariance Type:      

## Section 2

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import statsmodels.api as sm

# Load the data
file_path = '/Users/tangjiahong/Dropbox/HW_statistics/finalexam/GiftCards6.xlsx'
data = pd.read_excel(file_path)

# Combine 'Never married' and 'Divorced' into a single category
data['Marital_Status'] = data['Marital Status'].replace({'Never married': 'Not Married', 'Divorced': 'Not Married', 'Married': 'Married'})

# Encode categorical variables
data['Urban'] = LabelEncoder().fit_transform(data['Urban'])
data['Male'] = LabelEncoder().fit_transform(data['Male'])
data['Marital_Status'] = LabelEncoder().fit_transform(data['Marital_Status'])

# Display the first few rows to ensure data is correct
print(data.head())


  GiftCard  Spending  Income_1000 Marital Status  Urban  Age  Male  \
0      Yes       981          123        Married      1   37     0   
1      Yes       663          121        Married      1   36     0   
2      Yes       308           66        Married      1   26     1   
3      Yes       398          102        Married      1   36     1   
4      Yes       297           95        Married      0   34     1   

   Marital_Status  
0               0  
1               0  
2               0  
3               0  
4               0  


In [8]:
# Define the predictors and the response variable for the full model
X_full = data[['Income_1000', 'Urban', 'Male', 'Age', 'Marital_Status']]
y = data['Spending']

# Add a constant to the model (intercept)
X_full = sm.add_constant(X_full)

# Fit the full regression model
full_model = sm.OLS(y, X_full).fit()

# Display the model summary
print(full_model.summary())


                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     41.78
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           6.49e-33
Time:                        16:55:45   Log-Likelihood:                -2284.0
No. Observations:                 320   AIC:                             4580.
Df Residuals:                     314   BIC:                             4603.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           -284.0848    131.067     -2.

In [9]:
# Function for backward elimination
def backward_elimination(data, response, alpha=0.1):
    initial_list = data.columns.tolist()
    while len(initial_list) > 1:  # Ensure at least one predictor is left
        model = sm.OLS(response, sm.add_constant(data[initial_list])).fit()
        p_values = model.pvalues[1:]  # Exclude the intercept
        max_p_value = p_values.max()
        if max_p_value > alpha:
            excluded_feature = p_values.idxmax()
            initial_list.remove(excluded_feature)
            print(f"Excluding {excluded_feature} with p-value {max_p_value}")
        else:
            break
    return sm.OLS(response, sm.add_constant(data[initial_list])).fit()

# Perform backward elimination
final_model = backward_elimination(X_full, y)

# Display the final model summary
print(final_model.summary())


Excluding Marital_Status with p-value 0.2666452181208078
                            OLS Regression Results                            
Dep. Variable:               Spending   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.389
Method:                 Least Squares   F-statistic:                     51.87
Date:                Sat, 08 Jun 2024   Prob (F-statistic):           1.55e-33
Time:                        16:55:54   Log-Likelihood:                -2284.6
No. Observations:                 320   AIC:                             4579.
Df Residuals:                     315   BIC:                             4598.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

## Section 3 

In [12]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder

# Load the data
file_path = '/Users/tangjiahong/Dropbox/HW_statistics/finalexam/GiftCards6.xlsx'
data = pd.read_excel(file_path)

# Combine 'Never married' and 'Divorced' into a single category
data['Marital_Status'] = data['Marital Status'].replace({'Never married': 'Not Married', 'Divorced': 'Not Married', 'Married': 'Married'})

# Encode categorical variables
data['Urban'] = LabelEncoder().fit_transform(data['Urban'])
data['Male'] = LabelEncoder().fit_transform(data['Male'])
data['Marital_Status'] = LabelEncoder().fit_transform(data['Marital_Status'])
data['GiftCard'] = LabelEncoder().fit_transform(data['GiftCard'])

# Define the predictors and the response variable for the logistic model
X_logit = data[['Income_1000', 'Age', 'Urban', 'Male', 'Marital_Status']]
y_logit = data['GiftCard']

# Add a constant to the model (intercept)
X_logit = sm.add_constant(X_logit)

# Fit the logistic regression model
logit_model = sm.Logit(y_logit, X_logit).fit()

# Display the model summary
print(logit_model.summary())

# Calculate the adjusted odds ratios
odds_ratios = np.exp(logit_model.params)
print(f"Odds Ratios:\n{odds_ratios}\n")

# Estimate probability for specified customers
def estimate_probability(income_1000, age, urban, male, marital_status):
    logit = (logit_model.params['const'] + 
             logit_model.params['Income_1000'] * income_1000 + 
             logit_model.params['Age'] * age + 
             logit_model.params['Urban'] * urban + 
             logit_model.params['Male'] * male + 
             logit_model.params['Marital_Status'] * marital_status)
    probability = np.exp(logit) / (1 + np.exp(logit))
    return probability

# Probability for a female customer, age 40, annual income $60,000, living in rural, never married
prob_female_never_married = estimate_probability(income_1000=60, age=40, urban=0, male=0, marital_status=0)
print(f"Probability (Female, Never Married): {prob_female_never_married}")

# Probability for a female customer, age 40, annual income $60,000, living in rural, married
prob_female_married = estimate_probability(income_1000=60, age=40, urban=0, male=0, marital_status=1)
print(f"Probability (Female, Married): {prob_female_married}")

# Statistical test for the association between age and gift card purchase
p_value_age = logit_model.pvalues['Age']
print(f"P-value for Age: {p_value_age}")

# Null hypothesis: beta_2 (coefficient for Age) = 0
# Alternative hypothesis: beta_2 (coefficient for Age) ≠ 0
alpha = 0.05
if p_value_age < alpha:
    print("Reject the null hypothesis: There is an association between age and gift card purchase.")
else:
    print("Fail to reject the null hypothesis: There is no association between age and gift card purchase.")

# Adjusted odds ratio for living in urban areas
odds_ratio_urban = odds_ratios['Urban']
ci_urban = np.exp(logit_model.conf_int().loc['Urban'])
print(f"Adjusted Odds Ratio (Urban): {odds_ratio_urban}")
print(f"95% CI for Adjusted Odds Ratio (Urban): {ci_urban}")

# Test association using 95% CI for odds ratio
if ci_urban[0] > 1 or ci_urban[1] < 1:
    print("Reject the null hypothesis: There is an association between living in urban areas and the chance of purchasing gift cards.")
else:
    print("Fail to reject the null hypothesis: There is no association between living in urban areas and the chance of purchasing gift cards.")


Optimization terminated successfully.
         Current function value: 0.457434
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               GiftCard   No. Observations:                  320
Model:                          Logit   Df Residuals:                      314
Method:                           MLE   Df Model:                            5
Date:                Sat, 08 Jun 2024   Pseudo R-squ.:                  0.1763
Time:                        19:41:02   Log-Likelihood:                -146.38
converged:                       True   LL-Null:                       -177.72
Covariance Type:            nonrobust   LLR p-value:                 3.399e-12
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -1.3794      1.103     -1.250      0.211      -3.541       0.783
Income_1000      